In [ ]:
import json
import csv
import requests
import io
import time
import zipfile
import wikidata_parser

In [ ]:
# {"requested_QID":{"P31":{"valueID":"label"},"P279":{"valueID":"label"}}}

In [ ]:
headers = {"user-agent":"bot getting claims of the requested entities (CWI; Human-Centered Data Analytics; nesterov@cwi.nl)"}

In [ ]:
# importing EN results
with open('term_found_en.json','r') as jf:
    entities_en = json.load(jf)

In [ ]:
# importing NL results
with open('term_found_nl.json','r') as jf:
    entities_nl = json.load(jf)

In [ ]:
# putting all unique QIDs in a list

qids = []

for term, values in entities_en.items():
    for v in values:
        qids.append(v["QID"])
        
for term, values in entities_nl.items():
    for v in values:
        qids.append(v["QID"])

In [ ]:
unique_qids = list(set(qids))

In [ ]:
len(unique_qids)

#### Getting claims for every entity (language independent) in batches (2K)
- files with batches are not on GitHub; only the merged json file is available

In [ ]:
# Putting the query results in separate files (batches)
# Batch = 2K quids with claims
# [{quid:{'P31':'','P279':''}}]

n_batches = len(unique_qids) // 2000 + 1

start_batch_at = 0
end_batch_at = 0

for batch in range(0,n_batches):
    
    end_batch_at = end_batch_at + 2000
    
    qids_sublist = not_requested[start_batch_at:end_batch_at]

    # return a list of quids that were failed; [] if ok
    failed = wikidata_parser.request_claims(qids_sublist,['P31','P279'],headers,'/claims/',i)
    
    print(end_batch_at, failed)
    
    start_batch_at = start_batch_at + 2000
    
    time.sleep(1)

###### Merging files with claims

In [ ]:
import os
from os import listdir
from os.path import isfile,join

In [ ]:
dir_path = 'claims'
claims_paths = [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path,f)) and f.endswith('.json')]

In [ ]:
all_claims = []
for file in claims_paths:
    with open(file, "r") as jf:
        claims = json.load(jf)
        all_claims.extend(claims)

In [ ]:
# saving all claims in one file
with open("all_claims.json", "w") as f:
    json.dump(all_claims,f)

#### Getting labels of property values (EN and NL)

##### Getting unique value IDs of P31 and P279

In [ ]:
# using all_claims

In [ ]:
values_ids = []
for claim in all_claims:
    for value in claim.values():
        values_ids.extend(value['P31'].keys())
        values_ids.extend(value['P279'].keys())

In [ ]:
len(values_ids)

In [ ]:
unique_values_ids = list(set(values_ids))

In [ ]:
len(unique_values_ids)

In [ ]:
headers = {"user-agent":"bot getting labels of the requested entities (CWI; Human-Centered Data Analytics; nesterov@cwi.nl)"}

In [ ]:
# saves 'labels_property_values_1.json'
# retuns a list of failed entities
wikidata_parser.request_labels_of_property_values(unique_values_ids,headers,'')

#### Merging claims with the property labels

In [ ]:
# reading the property labels file
with open("labels_property_values_1.json","r") as jf:
    property_labels = json.load(jf)

In [ ]:
for claim in all_claims:
    for value in claim.values():
        for k in value['P31'].keys():
            value['P31'][k] = property_labels.get(k)
        for k in value['P279'].keys():
            value['P279'][k] = property_labels.get(k)

In [ ]:
# modifying claims
claims_dict = {}

for claim in all_claims:
    for qid, props in claim.items():
        claims_dict[qid] = props

In [ ]:
len(claims_dict)

In [ ]:
# saving all claims with labels (languge independent)
with open("all_claims_w_labels.json", "w") as f:
    json.dump(claims_dict,f)